In [12]:
from pathlib import Path
import os
import shutil
import cv2

DATASET_ROOT = Path("../../atlas_dione_objectdetection/ATLAS_Dione_ObjectDetection/ATLAS_Dione_ObjectDetection_Study_ActionClips/ATLAS_Dione_ObjectDetection_Study_ActionClips")
YOLO_DATASET_TEST = Path("../../YOLO_dataset_classification/test")
YOLO_DATASET_TRAIN = Path("../../YOLO_dataset_classification/train")
YOLO_DATASET_VAL = Path("../../YOLO_dataset_classification/val")
if(not os.path.exists("../../YOLO_dataset_classification")):
    os.mkdir("../../YOLO_dataset_classification")
    #creation directory for yolo dataset image classification
    os.mkdir("../../YOLO_dataset_classification/train")
    os.mkdir("../../YOLO_dataset_classification/test")
    os.mkdir("../../YOLO_dataset_classification/val")

In [13]:
def frame_video_extraction(video, output_dir):
    cap = cv2.VideoCapture(video)
    if not cap.isOpened():
        print("Errore nell'aprire il video.")
        exit()
    frame_count = 0
    while True:
        # Leggi il frame successivo
        ret, frame = cap.read()
        
        # Se non ci sono più frame, esci dal ciclo
        if not ret:
            break
        
        # Crea il nome del file per il frame, ad esempio frame_0001.jpg
        frame_filename = os.path.join(output_dir, f"{video.name[:-4]}_frame_{frame_count:04d}.jpg")
        
        # Salva il frame come immagine
        cv2.imwrite(frame_filename, frame)
        
        frame_count += 1

    # Rilascia il video quando finito
    cap.release()

In [14]:
mapping = {"set00":"1 arm placing",
            "set01":"2 arms placing",
            "set02":"Placing Rings",
            "set03":"Placing Rings 2 arms",
            "set04":"Pull Off",
            "set05":"Pull Through",
            "set06":"Suture Pick Up",
            "set07":"UVA Pick Up",
            "set08":"Suture Pull Through",
            "set09":"UVA Pull Through",
            "set10":"Suture Tie",
            "set11":"UVA Tie",
            }

In [ ]:
directory_index = "00"
total_directory = 12

for i in range(0,total_directory):
    sub_action="set" + directory_index
    directory_set_name = mapping[sub_action]
    set_directory = os.path.join(DATASET_ROOT,directory_set_name)
    videos = list(Path(os.path.join(DATASET_ROOT,sub_action)).glob("set*.mkv"))

    total_videos = len(videos)
    train_count = int(0.8 * total_videos)
    val_count = int(0.1 * total_videos)
    test_count = total_videos - train_count - val_count  # This will cover the remaining 10%
    # Make sure the counts sum up correctly, if necessar
    if val_count == 0 and test_count > 0:
        # If test_count is zero and val_count is greater than 0, move one video to test
        test_count -= 1
        val_count = 1

    yolo_train_set = os.path.join(YOLO_DATASET_TRAIN, directory_set_name)
    yolo_test_set = os.path.join(YOLO_DATASET_TEST, directory_set_name)
    yolo_val_set = os.path.join(YOLO_DATASET_VAL, directory_set_name)
    #create sub directory in yolo dataset
    if((not os.path.exists(yolo_test_set)) and (not os.path.exists(yolo_train_set)) and (not os.path.exists(yolo_val_set))):
        os.mkdir(os.path.join(YOLO_DATASET_TRAIN, directory_set_name))
        os.mkdir(os.path.join(YOLO_DATASET_TEST, directory_set_name))
        os.mkdir(os.path.join(YOLO_DATASET_VAL, directory_set_name))
    
        for video in videos[:train_count]:
            frame_video_extraction(video, yolo_train_set)
            #shutil.copy(video,yolo_train_set)
        for video in videos[train_count:train_count + val_count]:
            frame_video_extraction(video, yolo_test_set)

        for video in videos[train_count + val_count:]:
            frame_video_extraction(video, yolo_val_set)
            #shutil.copy(video, yolo_val_set)

    
    directory_index = str(int(directory_index) + 1).zfill(2)